# AKNet

In [1]:
import os
import pathlib
import tensorflow as tf
import pandas as pd
import wandb

from bcd.model.network.base import NetworkConfig
from bcd.model.network.simplenet import SimpleNetConfig, SimpleNetFactory
from bcd.model.store import ExperimentRepo
from bcd.model.pretrained import *
from bcd.model.experiment import FeatureExtractionExperiment
from bcd.model.config import *

pd.set_option('display.max_rows',999)

## Configuration

In [2]:
# Experiment Parameters
%env "WANDB_NOTEBOOK_NAME" "aknet.ipynb"
mode = "Stage"
force = True  # Whether to retrain if the model and config already exists.
base_models = [DenseNet(), EfficientNet(), Inception(), InceptionResNet(), MobileNet(), ResNet(), Xception()]

env: "WANDB_NOTEBOOK_NAME"="aknet.ipynb"


In [3]:
def create_config(network_config: NetworkConfig):
    project_config = ProjectConfig(mode=mode)

    train_config = TrainConfig(epochs=50, learning_rate=1e-4)    

    dataset_config = DatasetConfig(mode=mode)

    checkpoint_config = CheckPointConfig(monitor="val_accuracy", verbose=1, save_best_only=True, save_weights_only=False, mode="auto")

    early_stop_config = EarlyStopConfig(min_delta=1e-4, monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

    learning_rate_schedule_config = LearningRateScheduleConfig(min_delta=1e-4, monitor="val_loss", factor=0.5, patience=3, restore_best_weights=True, verbose=1, mode="auto")

    config = Config(project=project_config, 
                    dataset=dataset_config, 
                    train=train_config, 
                    network=network_config, 
                    checkpoint=checkpoint_config, 
                    early_stop=early_stop_config, 
                    learning_rate_schedule=learning_rate_schedule_config)
    return config

network_config = SimpleNetConfig(activation="sigmoid", input_shape=(224,224,3), output_shape=1)
config = create_config(network_config=network_config)

## Load Data

In [4]:
train_dir = pathlib.Path(config.dataset.train_dir).with_suffix('') 
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

# Validation DataSet (10%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

Found 816 files belonging to 2 classes.
Using 653 files for training.
Found 816 files belonging to 2 classes.
Using 163 files for validation.


## Callbacks

In [5]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=config.early_stop.monitor, 
                                                       min_delta=config.early_stop.min_delta,
                                                       patience=config.early_stop.patience, 
                                                       restore_best_weights=config.early_stop.restore_best_weights,
                                                       verbose=config.early_stop.verbose)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=config.learning_rate_schedule.monitor,
                                                          factor=config.learning_rate_schedule.factor,
                                                          patience=config.learning_rate_schedule.patience,
                                                          verbose=config.learning_rate_schedule.verbose,
                                                          mode=config.learning_rate_schedule.mode,
                                                          min_delta=config.learning_rate_schedule.min_delta,
                                                          min_lr=config.learning_rate_schedule.min_lr)
callbacks = [early_stop_callback, reduce_lr_callback]

## Dependencies

In [6]:
repo = ExperimentRepo(mode = mode, project=config.project.name)
optimizer=tf.keras.optimizers.Adam
metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

## Build Factory

In [7]:
factory = SimpleNetFactory(config=config.network)

## Run Experiments

In [8]:
for base_model in base_models:
    network = factory.create(base_model=base_model)
    experiment = FeatureExtractionExperiment(network=network, config=config, optimizer=optimizer, repo=repo, callbacks=callbacks, metrics=metrics, tags=[network.architecture, base_model.name], force=False)
    experiment.run(train_ds=train_ds, val_ds=val_ds)


# ================================================================================================ #
                                         SimpleNet_DenseNet                                         
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SimpleNet_DenseNet_input_la  [(None, 224, 224, 3)]    0         
 yer (InputLayer)                                                
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TF

wandb: Currently logged in as: aistudio. Use `wandb login --relogin` to force relogin


Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 0.6861 - accuracy: 0.5467 - auc: 0.5518 - precision: 0.4662 - recall: 0.2413
Epoch 1: val_accuracy improved from -inf to 0.57669, saving model to models/stage/SimpleNet_DenseNet-v2xd6fwc.keras
21/21 [==============================] - 146s 6s/step - loss: 0.6861 - accuracy: 0.5467 - auc: 0.5518 - precision: 0.4662 - recall: 0.2413 - val_loss: 0.6754 - val_accuracy: 0.5767 - val_auc: 0.5385 - val_precision: 0.4186 - val_recall: 0.2903 - lr: 1.0000e-04
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.6773 - accuracy: 0.5651 - auc: 0.5744 - precision: 0.5068 - recall: 0.2622
Epoch 2: val_accuracy did not improve from 0.57669
21/21 [==============================] - 120s 6s/step - loss: 0.6773 - accuracy: 0.5651 - auc: 0.5744 - precision: 0.5068 - recall: 0.2622 - val_loss: 0.6720 - val_accuracy: 0.5706 - val_auc: 0.5462 - val_precision: 0.3947 - val_recall: 0.2419 - lr: 1.0000e-04
Epoch 3/50
21/21 [===

wandb: Waiting for artifact run-v2xd6fwc-SimpleNet_DenseNet-v2xd6fwc.keras to be committed...



wandb: Committed artifact aistudio/Breast-Cancer-Detection-Stage/run-v2xd6fwc-SimpleNet_DenseNet-v2xd6fwc.keras:v0
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

# ================================================================================================ #
                                         SimpleNet_Xception                                         
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SimpleNet_Xception_input_la  [(None, 224, 224, 3)]    0         
 yer (InputLayer)                                                
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                      

Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 0.6886 - accuracy: 0.5467 - auc: 0.5391 - precision: 0.4659 - recall: 0.2356
Epoch 1: val_accuracy did not improve from 0.66871

Epoch 1: val_accuracy did not improve from 0.65644

Epoch 1: val_accuracy did not improve from 0.63190

Epoch 1: val_accuracy did not improve from 0.63190

Epoch 1: val_accuracy did not improve from 0.63190

Epoch 1: val_accuracy did not improve from 0.62577

Epoch 1: val_accuracy improved from -inf to 0.58896, saving model to models/stage/SimpleNet_Xception-q57tsuk1.keras
21/21 [==============================] - 81s 4s/step - loss: 0.6886 - accuracy: 0.5467 - auc: 0.5391 - precision: 0.4659 - recall: 0.2356 - val_loss: 0.6735 - val_accuracy: 0.5890 - val_auc: 0.5050 - val_precision: 0.3529 - val_recall: 0.0968 - lr: 1.0000e-04
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.6804 - accuracy: 0.5666 - auc: 0.5566 - precision: 0.5149 - recall: 0.1818
Epoch 2: val_accuracy 

wandb: Waiting for artifact run-q57tsuk1-SimpleNet_Xception-q57tsuk1.keras to be committed...



wandb: Committed artifact aistudio/Breast-Cancer-Detection-Stage/run-q57tsuk1-SimpleNet_Xception-q57tsuk1.keras:v0
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch/accuracy,▁▃▃▂▃▃▄▄▄▅▅▅▆▆▆▆▅▆▆▇▆▆▆▇▇▆▇▆▆█▇▆▆▅▆▇▆▅▆▆
epoch/auc,▁▂▂▂▃▃▄▅▄▅▅▅▅▆▆▇▆▇▆▇▆▆▆▇█▆▆▇▇█▇▇▇▇▇▇▇▇▇▇
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch/learning_rate,███████████▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/loss,█▇▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▃▂▃▃▃▂▁▂▂▂▂▁▂▂▂▂▂▂▂▃▂▂
epoch/lr,████████████▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch/precision,▁▃▃▂▃▄▄▅▄▆▇▅▅▆▅▆▆▇▇█▇▇▆█▇▇▇▇▆██▆▇▆▆▇▆▅▆▇
epoch/recall,▄▂▂▂▁▂▄▄▃▁▂▅▇▆▆▆▅▆▅▅▅▅▅▅▅▅▇▆▆█▆▅▆▄▆▇▆▅▇▇
epoch/val_accuracy,▆▁▆▅▆▄▃▆██▇▆▇▆▆▇█▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
epoch/val_auc,▁▂▂▂▃▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████
epoch/val_loss,▆█▆▆▅▆▅▄▂▃▃▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
